In [29]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import cv2
import gc
import matplotlib.pyplot as plt
from IPython.display import Image, display, Markdown
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from time import perf_counter
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.layers as tfl
from keras.optimizers import SGD   
from  sklearn.preprocessing import LabelEncoder 
from torch.utils.data import Dataset, DataLoader

In [30]:
df_original= pd.read_csv('/home/bear/prakash/aai541/finalproject/data/train.csv')
df_test = pd.read_csv('/home/bear/prakash/aai541/finalproject/data/test.csv')
df_other = pd.read_csv('/home/bear/prakash/aai541/finalproject/data/other.csv')
df_original.shape
print("Train size:", len(df_original), "Train Patient Samples:", len(df_original.patient_id.unique()))
print("Test size:", len(df_test), "Test Patient Samples:", len(df_test.patient_id.unique()))
print("Other size:", len(df_other), "Other Patient Samples:", len(df_other.patient_id.unique()))

Train size: 754 Train Patient Samples: 632
Test size: 4 Test Patient Samples: 4
Other size: 396 Other Patient Samples: 336


In [38]:
# its bombing after certain size . Removing the size limit
#Before
print(df_original.head())
# add images path to train and get width and height of images
dfX=pd.DataFrame(columns=['Filepath', 'Label'])
dfX["Filepath"] = "/home/bear/prakash/aai541/finalproject/data/train/" + df_original["image_id"] + ".tif"
dfX["Label"] = df_original.label
#After
dfX = dfX[:100]
print(dfX.head())

   image_id  center_id patient_id  image_num label
0  006388_0         11     006388          0    CE
1  008e5c_0         11     008e5c          0    CE
2  00c058_0         11     00c058          0   LAA
3  01adc5_0         11     01adc5          0   LAA
4  026c97_0          4     026c97          0    CE
                                            Filepath Label
0  /home/bear/prakash/aai541/finalproject/data/tr...    CE
1  /home/bear/prakash/aai541/finalproject/data/tr...    CE
2  /home/bear/prakash/aai541/finalproject/data/tr...   LAA
3  /home/bear/prakash/aai541/finalproject/data/tr...   LAA
4  /home/bear/prakash/aai541/finalproject/data/tr...    CE


/usr/lib/python3/dist-packages/PIL/Image.py:2896: DecompressionBombWarning: Image size (2934520020 pixels) exceeds limit of 1933120000 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [39]:
#8586586550
#
# This loads images using Image Data Generator
def create_gen(train_df, test_df):
    # Load the Images with a generator and Data Augmentation
    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        validation_split=0.1
    )

    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=True,
        seed=0,
        subset='training',
        rotation_range=30, # Uncomment to use data augmentation
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    val_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=True,
        seed=0,
        subset='validation',
        rotation_range=30, # Uncomment to use data augmentation
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )

    return train_generator,test_generator,train_images,val_images,test_images

In [33]:
sgd = SGD(learning_rate=0.001)
def get_model(model):
# Load the pretained model
    kwargs =    {'input_shape':(224, 224, 3),
                'include_top':False,
                'weights':'imagenet',
                'pooling':'avg'}
    
    pretrained_model = model(**kwargs)
    pretrained_model.trainable = False
    
    inputs = pretrained_model.input

    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(36, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=sgd,#'adam',
        loss= 'binary_crossentropy', #'categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [44]:
import PIL 

PIL.Image.MAX_IMAGE_PIXELS = 1933120000
# Dictionary with the models
models = {
    "EfficientNetB4": {"model":tf.keras.applications.efficientnet.EfficientNetB4, "perf":0},    
    "VGG16": {"model":tf.keras.applications.VGG16, "perf":0},
    "InceptionResNetV2": {"model":tf.keras.applications.InceptionResNetV2, "perf":0},
    "ResNet101": {"model":tf.keras.applications.ResNet101, "perf":0},
    "InceptionV3": {"model":tf.keras.applications.InceptionV3, "perf":0},
    "MobileNet": {"model":tf.keras.applications.MobileNet, "perf":0},
}

def buildTransferModels(TRAIN_IMAGES, VAL_IMAGES, name):
    print("Getting model:", name)
    model = models.get(name)
    print("processing:", name)
    # Get the model
    m = get_model(model['model'])
    models[name]['model'] = m
    start = perf_counter()
    print('fitting the model:', name)
    history = m.fit(TRAIN_IMAGES,
                    validation_data=VAL_IMAGES,
                    epochs=10,
                    verbose=2,
                   batch_size = 32)

    # Sav the duration, the train_accuracy and the val_accuracy
    duration = perf_counter() - start
    duration = round(duration,2)
    models[name]['perf'] = duration
    print(f"{name:20} trained in {duration} sec")
    
    val_acc = history.history['val_accuracy']
    models[name]['val_acc'] = [round(v,4) for v in val_acc]
    
    train_acc = history.history['accuracy']
    models[name]['train_accuracy'] = [round(v,4) for v in train_acc]
    m.save(name+"36.h5")

In [ ]:
## Splitting data
#X_train,X_val,Y_train,Y_val=train_test_split(images,labels, test_size=0.2, random_state=42)

In [45]:

#dfX = pd.DataFrame((images, labels))
# Concatenate filepaths and labels
#dfX = pd.concat([np.array(images), np.array(labels)], axis=1)
# Separate in train and test data
TRAIN_DF, TEST_DF = train_test_split(dfX, train_size=0.9, shuffle=True, random_state=1)
print(dfX.shape, TRAIN_DF.shape, TEST_DF.shape)

(100, 2) (90, 2) (10, 2)


In [46]:
# Create the generators
train_generator,test_generator,TRAIN_IMAGES,VAL_IMAGES,TEST_IMAGES=create_gen(TRAIN_DF, TEST_DF)

print('\n')

Found 81 validated image filenames belonging to 2 classes.
Found 9 validated image filenames belonging to 2 classes.
Found 10 validated image filenames belonging to 2 classes.




In [48]:

saved = buildTransferModels(TRAIN_IMAGES, VAL_IMAGES, "VGG16")#"EfficientNetB4") 
#saved = buildTransferModels(TRAIN_IMAGES, VAL_IMAGES)
print( "A total of :", saved," models from this build")

Getting model: VGG16
processing: VGG16
fitting the model: VGG16
Epoch 1/10


2023-08-03 10:45:43.730363: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 408.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-03 10:45:43.730963: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2023-08-03 10:45:43.816482: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.02GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-03 10:45:43.904870: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out o

ResourceExhaustedError: Exception encountered when calling layer 'block1_conv2' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[32,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

Call arguments received by layer 'block1_conv2' (type Conv2D):
  • inputs=tf.Tensor(shape=(32, 224, 224, 64), dtype=float32)